In [ ]:
# Ensure TensorFlow is installed with a specific version
!pip3 install tensorflow==2.7.0

In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
# Load dataset (modify this part based on your dataset)
# You can replace it with other datasets like MNIST or CIFAR-10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# Convert labels to One-Hot Encoding
train_labels_one_hot = to_categorical(train_labels, num_classes=10)
test_labels_one_hot = to_categorical(test_labels, num_classes=10)

In [ ]:
# Resize images to 32x32 and ensure 3 channels (RGB) if needed
train_images_resized = tf.image.resize(train_images, (32, 32))
test_images_resized = tf.image.resize(test_images, (32, 32))

In [ ]:
# If images have a single channel (like grayscale), use this:
# train_images_rgb = tf.image.grayscale_to_rgb(train_images_resized)
# test_images_rgb = tf.image.grayscale_to_rgb(test_images_resized)

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Build a model using VGG16 as a base model with custom layers
vgg_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg_model.trainable = False  # Freeze VGG16 layers

model = tf.keras.Sequential([
    vgg_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 classes for classification
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Define callbacks for training optimization
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')


In [ ]:
# Train the model
history = model.fit(datagen.flow(train_images_resized, train_labels_one_hot, batch_size=64),
                    steps_per_epoch=len(train_images_resized) // 64,
                    epochs=2,
                    validation_data=(test_images_resized, test_labels_one_hot),
                    callbacks=[early_stopping, model_checkpoint])

In [ ]:
# Save the final model
model.save('final_model.keras')

In [ ]:
# Plot training history (accuracy and loss)
import matplotlib.pyplot as plt

In [ ]:
# Plot accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()